In [55]:
from datetime import datetime, timedelta
from binance.client import Client
from time import sleep
import pandas as pd
import pickle
import time



def get_client():
    fn = 'C:\\Users\\FUJSC\\Documents\\key\\binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


def get_btc_futures_price(start_time, end_time):
    # Create a Binance Futures client instance
    client = get_client()

    # Convert the start_time and end_time strings to datetime objects
    start_datetime = datetime.strptime(start_time, '%d-%m-%Y %H:%M:%S')
    end_datetime = datetime.strptime(end_time, '%d-%m-%Y %H:%M:%S')

    # Initialize a list to store the klines
    klines = []

    # Define the interval as 15 minutes
    interval = "15m"

    # # Fetch the klines (candlestick data) from Binance Futures API
    # while start_datetime < end_datetime:
    #     # Calculate the end time as 15 minutes after the start time
    #     end_time = start_datetime + timedelta(minutes=15)

    #     # Convert the datetime objects to UNIX timestamps
    #     start_timestamp = int(start_datetime.timestamp() * 1000)
    #     end_timestamp = int(end_time.timestamp() * 1000)

    #     # Fetch the klines for the current interval
    #     current_klines = client.futures_klines(symbol="BTCUSDT", interval=interval,
    #                                            startTime=start_timestamp, endTime=end_timestamp)
    #     klines.extend(current_klines)

    #     # Update the start time for the next iteration
    #     start_datetime = end_time

    # Convert the datetime objects to UNIX timestamps
    start_timestamp = int(start_datetime.timestamp() * 1000)
    end_timestamp = int(end_datetime.timestamp() * 1000)

    current_klines = client.futures_klines(symbol="BTCUSDT", interval=interval, startTime=start_timestamp, endTime=end_timestamp)
    klines.extend(current_klines)

    # Extract the high and low prices from the klines
    high_price = float(max(klines, key=lambda x: float(x[2]))[2])  # High price is at index 2
    low_price = float(min(klines, key=lambda x: float(x[3]))[3])   # Low price is at index 3

    return high_price, low_price

def format_data():
    fn = 'C:\\Users\\FUJSC\\Downloads\\GRIND_BTC_15.csv'
    start_ts = int(time.time())
    raw = pd.read_csv(fn)
    entry_time = list(raw['DateTime'])
    signal = list(raw['Signal'])
    price = list(raw['Price'])
    exit_time = [entry_time[idx+1] for idx in range(len(entry_time)-1)]+[None]
    raw['entry_time'] = entry_time
    raw['exit_time'] = exit_time
    high_prices = [None]*len(entry_time)
    low_prices = [None]*len(entry_time)
    pnl=[None]*len(entry_time)
    tot = len(entry_time)
    for idx in range(tot):
        sleep(1)
        high_price, low_price = get_btc_futures_price(entry_time[idx], exit_time[idx])
        current_ts = int(time.time())
        elapsed_ts = (current_ts - start_ts)
        remaining = round(100 -(idx*100/tot), 2)
        eft = int((elapsed_ts*tot/idx)-elapsed_ts) if idx >0 else int((elapsed_ts*tot/1)-elapsed_ts)
        print(f'idx: {idx} \t Elapsed TS: {elapsed_ts}s \t Remaining: {remaining}% \t EFT: {eft}s')
        high_prices[idx] = high_price
        low_prices[idx] = low_price
        if signal[idx] == 'S':
            pnl[idx] = (round((price[idx]*100/low_price)-100, 2))
        if signal[idx] == 'L':
            pnl[idx] = (round((high_price*100/price[idx])-100, 2))
        raw['high_prices'] = high_prices
        raw['low_prices'] = low_prices
        raw['pnl'] = pnl
        raw.to_csv('formatted.csv', index=False)
    return raw





In [58]:
r = format_data()

idx: 0 	 Elapsed TS: 2s 	 Remaining: 100.0% 	 EFT: 204s
idx: 1 	 Elapsed TS: 4s 	 Remaining: 99.03% 	 EFT: 408s
idx: 2 	 Elapsed TS: 6s 	 Remaining: 98.06% 	 EFT: 303s
idx: 3 	 Elapsed TS: 9s 	 Remaining: 97.09% 	 EFT: 300s
idx: 4 	 Elapsed TS: 11s 	 Remaining: 96.12% 	 EFT: 272s
idx: 5 	 Elapsed TS: 13s 	 Remaining: 95.15% 	 EFT: 254s
idx: 6 	 Elapsed TS: 15s 	 Remaining: 94.17% 	 EFT: 242s
idx: 7 	 Elapsed TS: 17s 	 Remaining: 93.2% 	 EFT: 233s
idx: 8 	 Elapsed TS: 20s 	 Remaining: 92.23% 	 EFT: 237s
idx: 9 	 Elapsed TS: 22s 	 Remaining: 91.26% 	 EFT: 229s
idx: 10 	 Elapsed TS: 24s 	 Remaining: 90.29% 	 EFT: 223s
idx: 11 	 Elapsed TS: 26s 	 Remaining: 89.32% 	 EFT: 217s
idx: 12 	 Elapsed TS: 28s 	 Remaining: 88.35% 	 EFT: 212s
idx: 13 	 Elapsed TS: 30s 	 Remaining: 87.38% 	 EFT: 207s
idx: 14 	 Elapsed TS: 33s 	 Remaining: 86.41% 	 EFT: 209s
idx: 15 	 Elapsed TS: 35s 	 Remaining: 85.44% 	 EFT: 205s
idx: 16 	 Elapsed TS: 37s 	 Remaining: 84.47% 	 EFT: 201s
idx: 17 	 Elapsed TS: 39s 	 R

TypeError: strptime() argument 1 must be str, not None

In [57]:
start_time = "13-06-2023 16:00:00"
end_time = "13-06-2023 17:00:00"
high_price, low_price = get_btc_futures_price(start_time, end_time)
print(f"High Price: {high_price}")
print(f"Low Price: {low_price}")


High Price: 26430.5
Low Price: 25888.0
